<a href="https://colab.research.google.com/github/OSGeoLabBp/tutorials/blob/master/hungarian/python/regexp_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Szabályos kifejezések Pythonban

A szabályos kifejezés (regexp) egy hatékony eszköz különböző szövegminták kezelésére a szövegfeldolgozás során. Számos szövegszerkesztő (pl. Notepad++, vi) és programnyelv rendelkezik funkcióval a szabályos kifejezések kezelésére.

A szövegminták megadása során egyes karakterekhez speciális jelentés tartozik. Az alábbi rövid és nem teljes listában a legfontosabb speciális karaktereket soroljuk fel:

|karakter(ek)|magyarázat                                                       |
|------------|-----------------------------------------------------------------|
|. (pont)    |bármelyik karakter kivéve az újsor karaktert                     |
|^           |a sor/szöveg eleje                                               |
|$           |a sor/szöveg vége                                                |
|[abc]       |bármelyik karakter a szögletes zárójelben felsoroltak közül      |
|[^abc]      |egyik karakter sem a szögletes zárójelben felsoroltak közül      |
|[a-z]       |bármelyik karakter a szögletes zárójelben szereplő intervallumban|
|[^a-z]      |egyik karakter sem a szögletes zárójelben szereplő intervallumban|
|( )         |csoportképzés a mintában                                         |
|{min,max}   |az előző karakter vagy csoport ismétlése, max megadása opcionális|
|p1 \| p2    |p1 minta vagy p2 minta                                           |
|p\*         |a p karakter vagy csoport tetszőleges számú ismétlése, beleértve a 0-t is, egyenértékű a p{0,} mintával|
|p+          |a p karakter vagy csoport egy vagy többszöri ismétlése, egyenértékű a p{1,} mintával|
|p?          |a p karakter vagy csoport nulla vagy egy előfordulása, egyenértékű a p{0,1} mintával|
|\           |a következő karakter speciális jelentésének kikapcsolása (pl. \\. a pont karakter és nem bármelyik karakter jelenti)|


A Pythonban egy külön modul *re* áll rendelkezésre a szabályos kifejezések kezelésére. A használat előtt importálni kell:

In [1]:
import re

 Néhány példa következik a szabályos kifejezések kezelésére.

##Szöveglánc megfelel a szabályos kifejezésnek?



In [2]:
text = """Python egy általános célú, nagyon magas szintű programozási nyelv,
melyet Guido van Rossum holland programozó kezdett el fejleszteni 1989 végén,
majd hozott nyilvánosságra 1991-ben. A nyelv tervezési filozófiája az 
olvashatóságot és a programozói munka megkönnyítését helyezi előtérbe a futási
sebességgel szemben."""   # idézet a Wikipediaból

*re.match*  mintát csak a szöveg elején keresi. Találat esetén egy objektumot vany *None*-t vissza, ha nem találja.

In [3]:
re.match("Python", text)      # Python a szöveg elején szerepel?

<re.Match object; span=(0, 6), match='Python'>

In [4]:
if re.match("[Pp]ython", text): # Python vagy python van a szöveg elején?
    print('a szöveg Pythonnal kezdődik')

a szöveg Pythonnal kezdődik


In [5]:
result = re.match("[Pp]ython", text)
result.span(), result.group(0)


((0, 6), 'Python')

*re.search* a minta első előfordulását keresi meg a szövegben.

In [6]:
re.search('prog', text)

<re.Match object; span=(47, 51), match='prog'>

In [7]:
re.search('programozói?', text)        # opcionális 'i' a végén

<re.Match object; span=(99, 109), match='programozó'>

A minta összes előfordulásának megkeresése.

In [8]:
re.findall('programozói?', text)

['programozó', 'programozói']

Az *r* előtagot használjuk a szabályos kifejezésekhez.

In [12]:
re.findall(r'[ \t\r\n][a-záéíóöőúüű]{2}[ \t\r\n]', text) # kétbetűs szavak

[' el ', ' az ', ' és ']

In [11]:
re.findall(r'\s\w\w\s', text)   # ugyanaz mint az előző de rövidebb

[' el ', ' az ', ' és ']

In [14]:
re.findall(r'\sp\w*\s', text)    # 'p' betűvel kezdődő szavak

[' programozási ', ' programozó ', ' programozói ']

A szabályos kifejezést a keresés mellett az adatok formai ellenőrzésére is használhatjuk. Az alábbi példában ellenőrizzük, hogy a szövegek megfelelnek-e egy egész számnak.

In [15]:
int_numbers = ('12356', '1ac', 'tizenkettő', '23.65', '0', '-768')
for int_number in int_numbers:
  if re.match(r'[+-]?(0|[1-9][0-9]*)$', int_number):
    print(f'{int_number} egy egész szám')

12356 egy egész szám
0 egy egész szám
-768 egy egész szám


Készítsünk hasonló vizsgálatot lebegőpontos számokra.

In [16]:
float_numbers =('12', '0.0', '-43.56', '1.76e-1', '1.1.1', '00.289')
for float_number in float_numbers:
  if re.match(r'[+-]?(0|[1-9][0-9]*)(\.[0-9]*)?([eg][+-]?[0-9]+)?$', float_number):
    print(f'{float_number} egy lebegőpontos szám')

12 egy lebegőpontos szám
0.0 egy lebegőpontos szám
-43.56 egy lebegőpontos szám
1.76e-1 egy lebegőpontos szám


Egy másik megközelítés is alkalmazható a karakterláncban szereplő számértékre:

In [17]:
for float_number in float_numbers:
  try:
    float(float_number)     # try to convert to float number
  except ValueError:
    continue                # can't convert skip it
  print(f'{float_number} egy lebegőpontos szám')

12 egy lebegőpontos szám
0.0 egy lebegőpontos szám
-43.56 egy lebegőpontos szám
1.76e-1 egy lebegőpontos szám
00.289 egy lebegőpontos szám


Email cím ellenőrzés, egy lefordított szabályos kifejezést használunk (*re.compile*). Ez hatékonyabb, ha ugyanazt a mintát többször akarjuk használni.

In [18]:
email = re.compile(r'^[a-zA-Z0-9.!#$%&\'*+/=?^_`{|}~-]+@[a-zA-Z0-9]([a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?(\.[a-zA-Z0-9]([a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?)*$')
addresses = ['a.b@c', 'siki.zoltan@emk.bme.hu', 'plainaddress', '#@%^%#$@#$@#.com', '@example.com', 'Joe Smith <email@example.com>',
            'email.example.com', 'email@example@example.com', 'email@123.123.123.123']
valid_addresses = [addr for addr in addresses if email.search(addr)]
print('megfellő email címek:\n', valid_addresses)
invalid_addresses = [addr for addr in addresses if not email.search(addr)]
print('hibás email címek:\n', invalid_addresses)

megfelleő email címek:
 ['a.b@c', 'siki.zoltan@emk.bme.hu', 'email@123.123.123.123']
hibás email címek:
 ['plainaddress', '#@%^%#$@#$@#.com', '@example.com', 'Joe Smith <email@example.com>', 'email.example.com', 'email@example@example.com']


#További függvények

*re.sub* kicseréli a szabályos kifejézés előfordulását a megadott karakterlánccal a szövegben.

Szomszédos szóközök kicserélése egy szóközre.

In [27]:
print(re.sub(r'  *', ' ', 'Szöveg     sok     felesleges    szóközzel'))

Szöveg sok felesleges szóközzel


Elválasztó karakterek egységesítése vesszőre.

In [28]:
print(re.sub(r'[ \t,;]', ',', 'első,második;harmadik negyedik ötödik'))

első,második,harmadik,negyedik,ötödik


*re.split* a szöveg szétvágása egy listába, a szabályos kifejezésben megadott elválasztó karakterekkel.

In [30]:
words = re.split(r'[, \.\t\r\n]', text)   # szó elválasztó a szóköz, a pont, a tabulátor és a sorvége
words

['Python',
 'egy',
 'általános',
 'célú',
 '',
 'nagyon',
 'magas',
 'szintű',
 'programozási',
 'nyelv',
 '',
 'melyet',
 'Guido',
 'van',
 'Rossum',
 'holland',
 'programozó',
 'kezdett',
 'el',
 'fejleszteni',
 '1989',
 'végén',
 '',
 'majd',
 'hozott',
 'nyilvánosságra',
 '1991-ben',
 '',
 'A',
 'nyelv',
 'tervezési',
 'filozófiája',
 'az',
 '',
 'olvashatóságot',
 'és',
 'a',
 'programozói',
 'munka',
 'megkönnyítését',
 'helyezi',
 'előtérbe',
 'a',
 'futási',
 'sebességgel',
 'szemben',
 '']

Vegye észre, hogy az eredményben több üres karakterlánc '' szerepel, ahol egymás mellett. Javítsuk ki!

In [25]:
words = re.split(r'[, \.\t\r\n]+', text)  # szomszédos elválasztók összevonása
words

['Python',
 'egy',
 'általános',
 'célú',
 'nagyon',
 'magas',
 'szintű',
 'programozási',
 'nyelv',
 'melyet',
 'Guido',
 'van',
 'Rossum',
 'holland',
 'programozó',
 'kezdett',
 'el',
 'fejleszteni',
 '1989',
 'végén',
 'majd',
 'hozott',
 'nyilvánosságra',
 '1991-ben',
 'A',
 'nyelv',
 'tervezési',
 'filozófiája',
 'az',
 'olvashatóságot',
 'és',
 'a',
 'programozói',
 'munka',
 'megkönnyítését',
 'helyezi',
 'előtérbe',
 'a',
 'futási',
 'sebességgel',
 'szemben',
 '']

Miért van egy üres karakterlánc a lista végén?

##Komplex példa

Keressük meg Kipling Dzsungel könyvének angol változatában az 's'-el kezdődő  négybetűs szavak gyakoriságát.

In [34]:
import urllib.request
url = 'https://www.gutenberg.org/files/236/236-0.txt'
words = {}
with urllib.request.urlopen(url) as file:
  for line in file:
    ws = re.split(r'[, \.\t\r\n]+', line.decode('utf8'))
    for w in ws:
      w = w.lower()
      if re.match('[sS][a-z]{3}', w):
        if w in words:
          words[w] += 1
        else:
          words[w] = 1
print(f'{len(words.keys())} különböző s-el kezdődő négybetűs szó')
m = max(words, key=words.get)
print(f'a leggyakoribb: {m} {words[m]} előfordulás')


751 különböző s-el kezdődő négybetűs szó
a leggyakoribb: said 426 előfordulás


*Tasks*

*   Analyse and try to understand the used regular expressons for float and email
*   Create a regular expression for phone numbers
*   Which is the longest word in Kipling's book?
*   Are there words in the book with all the different vowels (aeiou) of the English ABC?
*   How could we handle plurals and other non-dictionary forms (e.g. Maugli's, sees, saw, seen, etc)

